<a href="https://colab.research.google.com/github/bxck75/A1_Colabs/blob/master/diamondpainting_img_2_canvas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!rm -r /content/diamondcanvas

In [18]:
import os
os.chdir('/content')

!git clone https://github.com/bxck75/diamond-art.git

fatal: destination path 'diamond-art' already exists and is not an empty directory.


In [19]:
os.chdir('/content/diamond-art')
!python setup.py install


running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based t

In [20]:
!mkdir -p /content/diamondcanvas
!mkdir -p /content/diamondcanvas/fractal
!mkdir -p /content/diamondcanvas/fractal/color_shifts

In [21]:
input_image = '/content/diamondcanvas/fractal/art_url.png'

In [22]:
!curl https://cdn.images.express.co.uk/img/dynamic/151/590x/Black-hole-picture-what-is-a-black-hole-survive-inside-event-horizon-singularity-1112326.webp -o {input_image}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 21578  100 21578    0     0   139k      0 --:--:-- --:--:-- --:--:--  139k


In [25]:
import numpy as np
from PIL import Image

def get_new_val(old_val, nc):
    """
    Get the "closest" colour to old_val in the range [0,1] per channel divided
    into nc values.

    """

    return np.round(old_val * (nc - 1)) / (nc - 1)

# For RGB images, the following might give better colour-matching.
#p = np.linspace(0, 1, nc)
#p = np.array(list(product(p,p,p)))
#def get_new_val(old_val):
#    idx = np.argmin(np.sum((old_val[None,:] - p)**2, axis=1))
#    return p[idx]

def fs_dither(img, nc):
    """
    Floyd-Steinberg dither the image img into a palette with nc colours per
    channel.

    """

    arr = np.array(img, dtype=float) / 255
    height, width = arr.shape[:2]

    for ir in range(height):
        for ic in range(width):
            # NB need to copy here for RGB arrays otherwise err will be (0,0,0)!
            old_val = arr[ir, ic].copy()
            new_val = get_new_val(old_val, nc)
            arr[ir, ic] = new_val
            err = old_val - new_val
            # In this simple example, we will just ignore the border pixels.
            if ic < width - 1:
                arr[ir, ic+1] += err * 7/16
            if ir < height - 1:
                if ic > 0:
                    arr[ir+1, ic-1] += err * 3/16
                arr[ir+1, ic] += err * 5/16
                if ic < width - 1:
                    arr[ir+1, ic+1] += err / 16

    carr = np.array(arr/np.max(arr, axis=(0,1)) * 255, dtype=np.uint8)
    return Image.fromarray(carr)


def palette_reduce(img, nc):
    """Simple palette reduction without dithering."""
    arr = np.array(img, dtype=float) / 255
    arr = get_new_val(arr, nc)

    carr = np.array(arr/np.max(arr) * 255, dtype=np.uint8)
    return Image.fromarray(carr)

def get_image(img_name, new_width=400, GREYSCALE=False):

    # Read in the image, convert to greyscale.
    img = Image.open(img_name)
    if GREYSCALE:
        img = img.convert('L')

    width, height = img.size
    new_height = int(height * new_width / width)
    img = img.resize((new_width, new_height), Image.ANTIALIAS)
    return img

colours = ['581845', '900c3f', 'c70039', 'ff5733', 'ffc30f']
colours = ['7d6608', '9d7d0a', 'b7950b', 'd4ac0d', 'f1c40f', 'f4d03f', 'f7dc6f', 'f9e79f']
def hex_to_tuple(s):
    return int(s[:2], 16), int(s[2:4], 16), int(s[4:], 16)
new_palette = [hex_to_tuple(s) for s in colours]

def mess_with_palette_dict(palette_dict):
    n = len(new_palette)
    for i, k in enumerate(palette_dict.keys()):
        palette_dict[k] = new_palette[i % n]

def random_mess_with_palette_dict(palette_dict):
    n = len(new_palette)
    for i, k in enumerate(palette_dict.keys()):
        palette_dict[k] = np.random.randint(0, 255, dtype=np.uint8, size=3)

def crazy_img(rimg, j):
    arr = np.array(rimg)
    height, width = arr.shape[:2]
    rarr = arr.reshape((width*height, 3))
    palette = set((tuple(t) for t in rarr))
    palette_arr = np.array(list(palette))
    offset = np.random.randint(0, 256, palette_arr.shape, dtype=np.uint8)
    new_palette_arr = palette_arr
    palette_len = len(palette)
    palette_dict = dict((tuple(palette_arr[i]), new_palette_arr[i]) for i in range(palette_len))
    random_mess_with_palette_dict(palette_dict)
    new_rarr = np.zeros_like(rarr)
    for i in range(width*height):
        new_rarr[i] = palette_dict[tuple(rarr[i])]
    new_rarr = new_rarr.reshape((height, width, 3))
    new_rimg = Image.fromarray(new_rarr)
    new_rimg.save(f'/content/diamondcanvas/fractal/color_shifts/img{j}.png')

img_name = input_image
img = get_image(img_name, GREYSCALE=False)
#r_img = fs_dither(img, 2)
r_img = palette_reduce(img, 8)

n = 4
for j in range(n):
    print(f'{j+1}/{n}')
    crazy_img(r_img, j+1)


input_image = '/content/diamondcanvas/fractal/color_shifts/img1.png'

FileNotFoundError: ignored

In [ ]:
import glob
for input_image in glob.glob('/content/diamondcanvas/fractal/*.png'):
  im = Image.open(input_image)
  # using Image.ADAPTIVE to avoid dithering
  clrs = 8
  res = [200,150]
  pall = im.getpalette()
  print(pall).reshape(256,3)
  out = im.convert('P', palette=Image.ADAPTIVE, colors=clrs)
  name, ext= os.path.splitext(input_image)
  output_image = input_image.replace(name, name + '_'+ str(clrs)+'_'+ str(res[0]) +'x'+ str(res[1]))
  resized = out.resize((res[0],res[1]))
  resized.save(output_image)
  cmd = "diamond_art " + output_image + " "+output_image.replace('drive/MyDrive/datasets/garbage_imgs_L','diamondcanvas/fractal').replace('.png','_canvas.png')+" -d 200 -g 2.8"
  os.system(cmd)

[212, 244, 107, 228, 216, 109, 118, 230, 143, 177, 181, 130, 100, 165, 37, 88, 163, 34, 47, 240, 146, 58, 205, 168, 74, 162, 190, 31, 175, 182, 200, 134, 146, 204, 75, 135, 63, 130, 125, 91, 69, 135, 234, 49, 86, 217, 19, 117, 100, 36, 122, 33, 30, 68, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
!diamond_art {output_image} {output_image.replace('drive/MyDrive/datasets/garbage_imgs_L','diamondcanvas/fractal').replace('.png','_canvas.png')} -d 200 -g 2.8

Traceback (most recent call last):
  File "/usr/local/bin/diamond_art", line 33, in <module>
    sys.exit(load_entry_point('diamond-art==0.1.0', 'console_scripts', 'diamond_art')())
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1130, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1055, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1404, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 760, in invoke
    return __callback(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/diamond_art-0.1.0-py3.10.egg/diamond_art/cli.py", line 35, in main
  File "/usr/local/lib/python3.10/dist-packages/diamond_art-0.1.0-py3.10.egg/diamond_art/diamond_art.py", line 23, in __init__
  File "/usr/local/lib/python3.10/dist-packages/PIL/Image.py", line 2975, in open
 

In [ ]:
!zip /content/fractal_canvas.zip /content/diamondcanvas/fractal/*_8_200x150_*.png

In [ ]:
!mkdir -p /content/diamondcanvas/fractal

In [ ]:
!mv /content/diamondcanvas/img*.png /content/diamondcanvas/fractal